<a href="https://colab.research.google.com/github/brunoalvesufu/cv/blob/master/Tech_Problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/content/Caminhonetes.zip
/content/.config/.last_update_check.json
/content/.config/active_config
/content/.config/config_sentinel
/content/.config/gce
/content/.config/.metricsUUID
/content/.config/configurations/config_default
/content/.config/logs/2019.08.22/16.14.16.878847.log
/content/.config/logs/2019.08.22/16.14.02.863676.log
/content/.config/logs/2019.08.22/16.14.17.917007.log
/content/.config/logs/2019.08.22/16.13.58.179258.log
/content/.config/logs/2019.08.22/16.14.13.419782.log
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_train_small.csv


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  7 13:59:26 2019

@author: vizentec

Image dataset rename and folder segmentation for keras model training
"""
def img_prep(srcfolder = 'data', destfolder = 'modeldata'):
# Function that copies and renames the images acording to it's class, adding an incremental ID to the name
# Such folders are placed inside a folder called "modeldata" by default
# The inputs are: 
# * srcfolder = name of the main folder containing the source data (default name: "data")
# * destfolder = name of the folder where the images will be placed (default name: "modeldata")
# * backupimg = not yet implemented!!! Variable responsable for indicating if the original dataset must be kept on
#               the original folder, default value keeps the backup, to delete set equal to 0(zero)
    
    # Create the folder tree
    import shutil
    
    try:
        shutil.copytree(srcfolder, destfolder)
        print("Directory '" + destfolder + "' created!")
    except FileExistsError:
        print("Directory '" + destfolder + "' already exists!")
        print("Delete the '" + destfolder + "' folder or choose another name!")
        return
    
    print("Images backed up to '" + destfolder + "'!")
    
    # Copy and rename the files to the tree
    import os
        
    for classfolder in os.listdir(destfolder):
        i = 0
        for filename in os.listdir("{}/{}".format(destfolder, classfolder)): 
            name = classfolder + "_" + str(i) + ".jpg"
            src = "{}/{}".format(destfolder, classfolder) +'/'+ filename
            dst = "{}/{}".format(destfolder, classfolder) +'/'+ name
            # rename the file 
            os.rename(src, dst)
            i += 1
    
    # Prints the name of each class and the total amount of images in it
    for classfolder in os.listdir(destfolder):
        imgquant = os.listdir("{}/{}".format(destfolder, classfolder))
        print("On class '{}' there are '{}' images".format(classfolder,len(imgquant)))

#%%
def img_to_folders(train_size, valid_size, test_size, srcfolder = 'modeldata'):
# Function that splits the imagens on folder 'srcfolder' into the folders test, valid and train
# * train_size = number of images to be placed on the train folder
# * valid_size = number of images to be placed on the valid folder
# * test_size = number of images to be placed on the test folder
# * srcfolder = name of the main folder containing the source data (default name: "modeldata")
    import random
    import os
    from random import seed
    from random import sample
        
    sample_size = train_size + valid_size + test_size
    
    # seed random number generator
    seed(random.randint(1, 1000))
    
    for imgclass in os.listdir(srcfolder):
        images_local = os.listdir(srcfolder+'/'+imgclass)
        # select a subset of images without replacement
        subset = sample(images_local, sample_size)
        
        try:
            os.makedirs(srcfolder+'/train/'+imgclass)    
            os.makedirs(srcfolder+'/valid/'+imgclass)   
            os.makedirs(srcfolder+'/test/'+imgclass)
            print("Folders train, validation and test were created for " + imgclass)    
        except FileExistsError:
            print("One or more of the folders already exists!") 
            
        for imgname in subset[:train_size]:    
            os.rename('{}/{}/{}'.format(srcfolder, imgclass, imgname), '{}/train/{}/{}'.format(srcfolder, imgclass, imgname))
        print('train folder created and ready!')
        for imgname in subset[train_size : train_size + valid_size]:
            os.rename('{}/{}/{}'.format(srcfolder, imgclass, imgname), '{}/valid/{}/{}'.format(srcfolder, imgclass, imgname))
        print('valid folder created and ready!')
        for imgname in subset[train_size + valid_size : train_size + valid_size + test_size]:
            os.rename('{}/{}/{}'.format(srcfolder, imgclass, imgname), '{}/test/{}/{}'.format(srcfolder, imgclass, imgname))
        print('test folder created and ready!')

#%%
def crop(image_path, coords, save_location):
    """
    @param image_path: The path to the image to edit
    @param coords: A tuple of x/y coordinates (x1, y1, width, height)
    @param save_location: Path to save the cropped image
    """
    from PIL import Image

    image_obj = Image.open(image_path)
    cropped_image = image_obj.crop(coords)
    cropped_image.save(save_location)

#%%
import time
start = time.time()
img_prep('/kaggle/input/tech-problems-images/techproblemsdata', 'modeldata')
img_to_folders(5000, 500, 400)
print("Task done, elapsed time:" + str(time.time() - start))


In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

base_dir = 'modeldata/'
train_dir = base_dir + 'train'
validation_dir = base_dir + 'valid'
test_dir = base_dir + 'test'

size_img = 250
size_conv = 3
size_batch = 5

In [0]:

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(size_img, size_img), 
                                                    batch_size=size_batch, class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(size_img, size_img), 
                                                              batch_size=size_batch, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(size_img, size_img), 
                                                  batch_size=size_batch, class_mode='categorical')

In [0]:
from keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(size_img, size_img, 3))

conv_base.trainable = True

In [0]:

from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, activation='softmax'))

from keras import optimizers
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-5), metrics=['acc'])


In [0]:

history = model.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=50)

In [0]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.show()

In [0]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test_acc:', test_acc)

In [0]:
model.save('tech_problems_noprep_82p.h5')
model_json = model.to_json()
with open("tech_problems_noprep_82p.json", "w") as json_file:
    json_file.write(model_json)